In [5]:
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from io import StringIO

In [3]:
# years we are scraping from
years = list(range(1989, 2025))

# webscraping url
url = f"https://www.basketball-reference.com/leagues/NBA_YEAR_standings.html"

In [4]:
for year in years:
    url = f"https://www.basketball-reference.com/leagues/NBA_{year}_standings.html"
    data = requests.get(url)
    with open("team/{}.html".format(year), "w+", encoding="utf-8") as file:
        file.write(data.text)
    # i hate basketballreference
    time.sleep(5)

In [6]:
teamdfs = []
for year in years:
    with open("team/{}.html".format(year), encoding="utf-8") as file:
        page = file.read()
    # two different tables, one for each conference
    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_='thead').decompose()
    team_table = soup.find(id="divs_standings_E")
    team = pd.read_html(StringIO(str(team_table)))[0]
    team["Year"] = year
    team["Team"] = team["Eastern Conference"]
    del team["Eastern Conference"]
    teamdfs.append(team)

    soup = BeautifulSoup(page, "html.parser")
    soup.find('tr', class_='thead').decompose()
    team_table = soup.find(id="divs_standings_W")
    team = pd.read_html(StringIO(str(team_table)))[0]
    team["Year"] = year
    team["Team"] = team["Western Conference"]
    del team["Western Conference"]
    teamdfs.append(team)
    

In [7]:
teams = pd.concat(teamdfs)
teams.head()

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,52,30,.634,—,116.7,112.9,3.62,1989,New York Knicks*
1,46,36,.561,6.0,111.9,110.4,1.68,1989,Philadelphia 76ers*
2,42,40,.512,10.0,109.2,108.1,1.26,1989,Boston Celtics*
3,40,42,.488,12.0,108.3,110.4,-1.77,1989,Washington Bullets
4,26,56,.317,26.0,103.7,110.1,-5.69,1989,New Jersey Nets


In [8]:
teams.to_csv("team.csv")